In [123]:
import json
from parsivar import FindStems
from preprocessor.utils import Normalizer, Tokenizer

## Load Data

In [124]:
def read_json(path):
  file = open(path)
  data = json.load(file)
  return data

In [125]:
PATH = "D:\Data\Docs\\University\\term7\Inforamtion Retrieval\project\search_engine\Dataset\IR_data_news_12k.json"
input_data = read_json(PATH)

In [126]:
print(list(input_data.values())[0])

{'title': 'اعلام زمان قرعه کشی جام باشگاه های فوتسال آسیا', 'content': '\nبه گزارش خبرگزاری فارس، کنفدراسیون فوتبال آسیا (AFC) در نامه ای رسمی به فدراسیون فوتبال ایران و باشگاه گیتی پسند زمان\xa0 قرعه کشی جام باشگاه های فوتسال آسیا را رسماً اعلام کرد. بر این اساس 25 فروردین ماه 1401 مراسم قرعه کشی جام باشگاه های فوتسال آسیا در مالزی برگزار می شود. باشگاه گیتی پسند بعنوان قهرمان فوتسال ایران در سال 1400 به این مسابقات راه پیدا کرده است. پیش از این گیتی پسند تجربه 3 دوره حضور در جام باشگاه های فوتسال آسیا را داشته که هر سه دوره به فینال مسابقات راه پیدا کرده و یک عنوان قهرمانی و دو مقام دومی بدست آورده است. انتهای پیام/\n\n\n', 'tags': ['اعلام زمان', 'قرعه\u200cکشی', 'قرعه\u200cکشی جام', 'قرعه\u200cکشی جام باشگاه\u200cهای فوتسال', 'ای اف سی', 'گیتی پسند'], 'date': '3/15/2022 5:59:27 PM', 'url': 'https://www.farsnews.ir/news/14001224001005/اعلام-زمان-قرعه-کشی-جام-باشگاه-های-فوتسال-آسیا', 'category': 'sports'}


In [127]:
print(list(input_data.values())[0].keys())

dict_keys(['title', 'content', 'tags', 'date', 'url', 'category'])


In [128]:
contents = [input_data[i]['content'] for i in input_data]

In [129]:
print(len(contents))

12202


In [130]:
print(contents[0])


به گزارش خبرگزاری فارس، کنفدراسیون فوتبال آسیا (AFC) در نامه ای رسمی به فدراسیون فوتبال ایران و باشگاه گیتی پسند زمان  قرعه کشی جام باشگاه های فوتسال آسیا را رسماً اعلام کرد. بر این اساس 25 فروردین ماه 1401 مراسم قرعه کشی جام باشگاه های فوتسال آسیا در مالزی برگزار می شود. باشگاه گیتی پسند بعنوان قهرمان فوتسال ایران در سال 1400 به این مسابقات راه پیدا کرده است. پیش از این گیتی پسند تجربه 3 دوره حضور در جام باشگاه های فوتسال آسیا را داشته که هر سه دوره به فینال مسابقات راه پیدا کرده و یک عنوان قهرمانی و دو مقام دومی بدست آورده است. انتهای پیام/



## Preprocessing

In [131]:
normalizer = Normalizer()
tokenizer = Tokenizer()
stemmer = FindStems()

def preprocess(contents):

  preprocessed_docs = []
  for content in contents:
    # normalizing
    normalized_content = normalizer.normalize(content)
    content_tokens = tokenizer.tokenize(normalized_content)
    tokens = []
    for token in content_tokens:
        token = stemmer.convert_to_stem(token)
          
        tokens.append(token)
    preprocessed_docs.append(tokens)
    # tokens of each doc
  return preprocessed_docs


In [132]:
preprocessed_docs = preprocess(contents)

In [133]:
print(len(preprocessed_docs))

12202


## Positional indexing

In [122]:
import heapq

In [93]:
class Term:
    def __init__(self):
        self.total_freq = 0
        self.pos_in_doc = {} 
        self.freq_in_doc = {}
        self.weight_in_doc = {}
        self.champ_list = {}

    def update_posting(self, doc_id, term_position):
      if doc_id not in self.pos_in_doc:
            self.pos_in_doc[doc_id] = []
            self.freq_in_doc[doc_id] = 0
      self.pos_in_doc[doc_id].append(term_position)
      self.freq_in_doc[doc_id] += 1
      self.total_freq += 1
    
    def get_docs(self):
      return self.pos_in_doc.keys()
    
    def get_weight_in_doc(self, doc_id):
      return self.weight_in_doc[doc_id]
    
    def calc_weight(self, doc_id, collection_size):
      self.weight_in_doc [doc_id] = calculate_tf_idf(self, doc_id, collection_size)
    
    def create_champ_list(self, r):
      self.champ_list = dict(sorted(self.weight_in_doc.items(), key=lambda item: item[1], reverse=True)[:r])

    def get_champ_list(self):
      return self.champ_list         


In [94]:
def positional_indexing(preprocessed_docs):
    p_inv_index = {} 
    for doc_id in range(len(preprocessed_docs)):
        for pos in range(len(preprocessed_docs[doc_id])):
            term = preprocessed_docs[doc_id][pos]
            if term in p_inv_index:
                term_obj = p_inv_index[term]
            else:
                term_obj = Term()
            term_obj.update_posting(doc_id, pos)
            p_inv_index[term] = term_obj

    return p_inv_index

In [95]:
def delete_the_k_most_repeated_terms(positional_index, k=50):
    max_heap = []
    for term in positional_index.keys():
        heapq.heappush(max_heap, (-positional_index[term].total_freq, term))

    k_most_repeated = []
    for _ in range(k):
        item = heapq.heappop(max_heap)
        positional_index.pop(item[1])
        k_most_repeated.append(item)
    
    print([(-key, value) for key, value in k_most_repeated])
    
    return positional_index


In [96]:
positional_index = positional_indexing(preprocessed_docs)
positional_index = delete_the_k_most_repeated_terms(positional_index)

[(219070, 'و'), (164285, 'در'), (133171, 'به'), (92913, 'از'), (82972, 'این'), (76219, 'که'), (68983, 'با'), (66735, 'را'), (58187, 'اس'), (43015, 'کرد&کن'), (31007, 'برای'), (30969, 'داشت&دار'), (28802, 'شد&شو'), (27640, 'تیم'), (26283, 'ان'), (24777, 'کرد'), (23942, 'بود&باش'), (22370, 'هم'), (21716, 'کشور'), (19787, 'شد'), (19707, 'ما'), (18729, 'یک'), (17776, 'بازی'), (17254, 'می'), (16196, 'باید'), (15862, 'تا'), (15844, 'های\u200c'), (15695, 'بر'), (15480, 'وی'), (14830, 'شده'), (14554, 'خود'), (14553, 'داد&ده'), (14509, 'مجلس'), (14416, 'اسلامی'), (14375, 'خواست&خواه'), (14038, 'گزارش'), (13861, 'فارس'), (13584, 'گفت'), (13190, 'مردم'), (13127, 'پیام'), (12772, 'رییس'), (12705, 'ایران'), (12427, 'خبرگزاری'), (12251, 'انتهای'), (12227, 'دولت'), (12204, 'اما'), (12117, 'گرفت&گیر'), (11952, 'سال'), (11022, 'بازیکن'), (10779, 'داشت&دارد')]


In [97]:
preprocessed_docs[10]

['به',
 'گزارش',
 'خبرگزاری',
 'فارس',
 'و',
 'به',
 'نقل',
 'از',
 'سایت',
 'باشگاه',
 'تراکتور',
 'میر',
 'معصوم',
 'سهراب',
 'مدیرعامل',
 'این',
 'باشگاه',
 'به',
 'همراه',
 'نمایندگانی',
 'از',
 'استانداری',
 'اذربایجان',
 'شرقی',
 'معاون',
 'عملیات',
 'نیرو',
 'انتظامی',
 'استان',
 'نمایندگانی',
 'از',
 'یگان',
 'ویژه',
 'استان',
 'مدیرکل',
 'ورزش',
 'و',
 'جوان',
 'استان',
 'نماینده',
 'اداره',
 'ورزش',
 'و',
 'جوان',
 'شهرستان',
 'تبریز',
 'و',
 'مدیر',
 'ورزشگاه',
 'یادگار',
 'امام',
 'امروز',
 'سه',
 'شنبه',
 '۲۴',
 'اسفند',
 'از',
 'بخش',
 'مختلف',
 'ورزشگاه',
 'یادگار',
 'امام',
 'ره',
 'تبریز',
 'بازدید',
 'کرده',
 'و',
 'اخرین',
 'وضعیت',
 'و',
 'نواقصات',
 'این',
 'مجموعه',
 'برای',
 'میزبانی',
 'از',
 'هوادار',
 'تراکتور',
 'را',
 'بررسی',
 'کرد&کن',
 'طی',
 'این',
 'بازدید',
 'تمهیدات',
 'لازم',
 'در',
 'راستا',
 'اماده\u200cسازی',
 'این',
 'ورزشگاه',
 'برای',
 'میزبانی',
 'از',
 'بازی',
 'تیم',
 'تراکتور',
 'اندیشیده',
 'شد',
 'انتهای',
 'پیام']

In [98]:
print(positional_index['ورزش'].pos_in_doc)

{3: [63], 10: [35, 41], 13: [18, 55, 484, 489, 522, 590, 640, 655, 681], 29: [519], 56: [28], 62: [13, 19, 34, 144, 180, 184, 200, 328], 65: [476], 70: [216, 260], 74: [24], 77: [47], 85: [58, 507], 86: [13, 23, 28, 36, 49, 57, 65, 74, 76, 99, 126, 167, 251, 296, 326, 360, 368, 386, 448, 523, 542, 543, 548, 574, 616, 641, 752, 982], 89: [433], 95: [65, 140, 162], 105: [6, 13, 73, 92, 150, 325, 481, 613, 637, 663], 106: [24, 50, 114, 136, 208, 228, 297, 374, 412, 431, 482, 610], 112: [135], 133: [18, 25, 46, 139], 140: [681, 783, 1244, 1269], 142: [1065], 144: [11, 41, 155, 247, 253, 292, 326, 370, 388, 424, 426, 441], 154: [5, 64, 73, 112, 146, 165, 198, 216, 226, 245, 279, 362, 370, 464, 471, 483, 568, 579, 625, 693, 701, 709, 818, 1034, 1064, 1079, 1195, 1241, 1297, 1397, 1416, 1422, 1429, 1451, 1560, 1739, 1768, 1801, 1895, 1931, 1952, 2035], 160: [143, 156, 167], 181: [282], 184: [40, 47, 112, 450, 465, 490, 492, 552, 715, 737, 747, 782, 822], 229: [258], 284: [154, 185, 204], 302:

In [99]:
print(positional_index['ورزش'].total_freq)
print(sum(positional_index['ورزش'].freq_in_doc.values()))

2669
2669


## Answering Query

In [100]:
from itertools import permutations
import re
import math
import numpy as np
     

In [101]:
def calculate_tf_for_query(term, tokens):
    freq = 0
    for token in tokens:
        if token == term:
            freq += 1
    if freq > 0:
        return 1 + math.log10(freq)
    return 0

In [102]:
def calculate_tf(term, doc_id):
    freq = term.freq_in_doc[doc_id]
    if freq > 0:  
        return 1 + math.log10(freq)
    return 0

In [103]:
def calculate_idf(term, collection_size):
  n = len (term.freq_in_doc)
  return math.log10(collection_size/n)

In [104]:
def calculate_tf_idf(term, doc_id, collection_size):
  return calculate_tf(term, doc_id) * calculate_idf(term, collection_size)

In [105]:
def calculate_weights(dictionary, collection_size):
  for term in dictionary:
    postings_list = dictionary[term].get_docs()
    for doc_id in postings_list:
        dictionary[term].calc_weight(doc_id, collection_size)

In [106]:
calculate_weights(positional_index, len(contents))

In [107]:
print(positional_index['ورزش'].pos_in_doc)

{3: [63], 10: [35, 41], 13: [18, 55, 484, 489, 522, 590, 640, 655, 681], 29: [519], 56: [28], 62: [13, 19, 34, 144, 180, 184, 200, 328], 65: [476], 70: [216, 260], 74: [24], 77: [47], 85: [58, 507], 86: [13, 23, 28, 36, 49, 57, 65, 74, 76, 99, 126, 167, 251, 296, 326, 360, 368, 386, 448, 523, 542, 543, 548, 574, 616, 641, 752, 982], 89: [433], 95: [65, 140, 162], 105: [6, 13, 73, 92, 150, 325, 481, 613, 637, 663], 106: [24, 50, 114, 136, 208, 228, 297, 374, 412, 431, 482, 610], 112: [135], 133: [18, 25, 46, 139], 140: [681, 783, 1244, 1269], 142: [1065], 144: [11, 41, 155, 247, 253, 292, 326, 370, 388, 424, 426, 441], 154: [5, 64, 73, 112, 146, 165, 198, 216, 226, 245, 279, 362, 370, 464, 471, 483, 568, 579, 625, 693, 701, 709, 818, 1034, 1064, 1079, 1195, 1241, 1297, 1397, 1416, 1422, 1429, 1451, 1560, 1739, 1768, 1801, 1895, 1931, 1952, 2035], 160: [143, 156, 167], 181: [282], 184: [40, 47, 112, 450, 465, 490, 492, 552, 715, 737, 747, 782, 822], 229: [258], 284: [154, 185, 204], 302:

In [108]:
positional_index['ورزش'].weight_in_doc

{3: 1.1264361823279525,
 10: 1.4655272614098875,
 13: 2.2013294716758307,
 29: 1.1264361823279525,
 56: 1.1264361823279525,
 62: 2.143709419573758,
 65: 1.1264361823279525,
 70: 1.4655272614098875,
 74: 1.1264361823279525,
 77: 1.1264361823279525,
 85: 1.4655272614098875,
 86: 2.7565673503783175,
 89: 1.1264361823279525,
 95: 1.6638828270018915,
 105: 2.252872364655905,
 106: 2.342064985165762,
 112: 1.1264361823279525,
 133: 1.8046183404918228,
 140: 1.8046183404918228,
 142: 1.1264361823279525,
 144: 2.342064985165762,
 154: 2.9549229159703216,
 160: 1.6638828270018915,
 181: 1.1264361823279525,
 184: 2.3812222794300673,
 229: 1.1264361823279525,
 284: 1.6638828270018915,
 302: 2.0029739060838265,
 315: 1.8046183404918228,
 319: 2.0783851922144474,
 324: 1.1264361823279525,
 339: 1.91378128557397,
 341: 1.1264361823279525,
 351: 1.1264361823279525,
 367: 1.6638828270018915,
 371: 1.1264361823279525,
 378: 2.2994985829018395,
 397: 1.1264361823279525,
 418: 1.4655272614098875,
 425: 1

In [109]:
positional_index['ورزش'].freq_in_doc

{3: 1,
 10: 2,
 13: 9,
 29: 1,
 56: 1,
 62: 8,
 65: 1,
 70: 2,
 74: 1,
 77: 1,
 85: 2,
 86: 28,
 89: 1,
 95: 3,
 105: 10,
 106: 12,
 112: 1,
 133: 4,
 140: 4,
 142: 1,
 144: 12,
 154: 42,
 160: 3,
 181: 1,
 184: 13,
 229: 1,
 284: 3,
 302: 6,
 315: 4,
 319: 7,
 324: 1,
 339: 5,
 341: 1,
 351: 1,
 367: 3,
 371: 1,
 378: 11,
 397: 1,
 418: 2,
 425: 1,
 435: 4,
 449: 9,
 452: 1,
 454: 3,
 456: 2,
 457: 1,
 458: 13,
 459: 1,
 460: 1,
 461: 1,
 465: 11,
 468: 2,
 469: 2,
 472: 12,
 478: 3,
 479: 2,
 495: 1,
 501: 8,
 505: 2,
 508: 1,
 510: 1,
 524: 1,
 525: 1,
 530: 2,
 531: 2,
 532: 1,
 533: 1,
 534: 9,
 535: 17,
 536: 2,
 538: 6,
 542: 1,
 544: 3,
 545: 3,
 561: 2,
 562: 4,
 563: 7,
 568: 4,
 569: 5,
 570: 5,
 576: 3,
 577: 9,
 592: 1,
 616: 1,
 635: 1,
 641: 1,
 655: 5,
 674: 1,
 683: 1,
 695: 6,
 707: 13,
 708: 3,
 709: 2,
 711: 1,
 714: 1,
 715: 1,
 716: 4,
 728: 3,
 730: 3,
 733: 1,
 743: 1,
 754: 1,
 756: 3,
 781: 3,
 786: 1,
 789: 1,
 790: 4,
 797: 2,
 800: 1,
 818: 1,
 819: 9,
 821

In [110]:
positional_index['فوتبال'].weight_in_doc

{0: 0.7369968554144593,
 1: 0.7369968554144593,
 2: 0.5664718399043003,
 3: 1.0451960815306944,
 7: 0.5664718399043003,
 9: 0.5664718399043003,
 12: 0.7369968554144593,
 15: 0.5664718399043003,
 16: 0.5664718399043003,
 23: 0.7369968554144593,
 29: 0.7369968554144593,
 33: 0.9075218709246182,
 35: 0.7369968554144593,
 36: 0.7369968554144593,
 37: 0.7369968554144593,
 40: 0.5664718399043003,
 42: 0.5664718399043003,
 45: 0.5664718399043003,
 48: 0.5664718399043003,
 53: 1.0451960815306944,
 63: 0.5664718399043003,
 64: 0.5664718399043003,
 66: 1.0072726104329548,
 69: 0.7369968554144593,
 74: 0.5664718399043003,
 76: 0.5664718399043003,
 79: 0.5664718399043003,
 80: 0.8367475949227957,
 81: 1.1974893802347184,
 82: 0.5664718399043003,
 84: 0.5664718399043003,
 85: 1.1070233499412911,
 86: 0.7369968554144593,
 87: 0.7369968554144593,
 88: 0.5664718399043003,
 89: 1.1329436798086006,
 90: 0.5664718399043003,
 91: 0.5664718399043003,
 94: 0.5664718399043003,
 95: 0.5664718399043003,
 96: 0

In [111]:
def champions_list(dictionary, r):
    for term in dictionary:
        dictionary[term].create_champ_list(r)

In [112]:
champions_list(positional_index, r=50)

In [113]:
positional_index['فوتبال'].get_champ_list()

{3457: 1.5523382947229012,
 4918: 1.4739937108289185,
 2098: 1.4411429059248357,
 163: 1.4190969174550954,
 6323: 1.403219434827096,
 6128: 1.3483226414532725,
 1950: 1.3378523126220876,
 5268: 1.3269164858389186,
 5395: 1.3269164858389186,
 1322: 1.3034686953187595,
 2286: 1.3034686953187595,
 2287: 1.2908497450202987,
 2788: 1.2908497450202987,
 6350: 1.2908497450202987,
 2626: 1.2775483654514501,
 5353: 1.2775483654514501,
 536: 1.2634865043057129,
 2109: 1.2634865043057129,
 5252: 1.2634865043057129,
 5303: 1.2634865043057129,
 6327: 1.2634865043057129,
 6673: 1.2634865043057129,
 1643: 1.2485719019449362,
 4926: 1.2485719019449362,
 5102: 1.2485719019449362,
 1276: 1.2326944193169371,
 1572: 1.2326944193169371,
 2268: 1.2326944193169371,
 2391: 1.2326944193169371,
 3057: 1.2326944193169371,
 3109: 1.2326944193169371,
 4962: 1.2326944193169371,
 5583: 1.2326944193169371,
 5991: 1.2326944193169371,
 6069: 1.2326944193169371,
 142: 1.2157210970408534,
 246: 1.2157210970408534,
 1974:

In [114]:
def get_docs_norm(dictionary, collection_size):
    lengths = np.zeros(collection_size)
    for term in dictionary:
        for doc_id, weight in dictionary[term].weight_in_doc.items():
            lengths[int(doc_id)] += weight ** 2
    return np.sqrt(lengths)

In [115]:
docs_norms = get_docs_norm(positional_index, len(contents))
print(docs_norms)

[10.53434085  7.98068347  7.65672157 ... 15.78370391 18.4344583
  8.2223567 ]


In [116]:
def cosine_similarity(query_tokens, dictionary, collection_size, norms):
  scores = {}
  for term in query_tokens:
    w_t_q = calculate_tf_for_query(term, query_tokens) * calculate_idf(dictionary[term], collection_size)
    for doc_id, w_t_d in dictionary[term].get_champ_list().items():
      if doc_id not in scores:
        scores[doc_id] = 0
      scores[doc_id] += w_t_q * w_t_d 
      
  for doc_id in scores:
    scores[doc_id] /= norms[int(doc_id)]
  return scores

In [118]:
def print_result(socres, k):
    sorted_scores = dict(sorted(socres.items(), key=lambda item: item[1],reverse=True)[:k])
    for doc_id in sorted_scores:
        title = input_data[str(doc_id)]['title']
        url = input_data[str(doc_id)]['url']
        print(doc_id, 'title: ', title, '\nurl: ', url)
        print('------------')

In [120]:
query = 'فوتبال'
query_tokens = preprocess([query])[0]
scores = cosine_similarity(query_tokens, positional_index, len(contents), docs_norms)
print_result(scores, k=5)

81 title:  ماجدی: فوتبال کشور به تغییرات نیاز دارد 
url:  https://www.farsnews.ir/news/14001223000539/ماجدی-فوتبال-کشور-به-تغییرات-نیاز-دارد
------------
1466 title:  نکونام: نفتی ها بهترین بازی خود را انجام دادند/بازیکنان جدیدمان کیفیت بالای خود را نشان دادند 
url:  https://www.farsnews.ir/news/14001204001165/نکونام-نفتی-ها-بهترین-بازی-خود-را-انجام-دادند-بازیکنان-جدیدمان-کیفیت
------------
139 title:  تقدیر مربی تیم فوتبال خلیج فارس از ماجدی بابت رسیدگی به شائبه تبانی در لیگ جوانان 
url:  https://www.farsnews.ir/news/14001222000297/تقدیر-مربی-تیم-فوتبال-خلیج-فارس-از-ماجدی-بابت-رسیدگی-به-شائبه-تبانی-در
------------
2268 title:  عربشاهی: تصمیم هیات رئیسه برای برکناری عزیزی خادم جهادی بود/ فعالیت  کمیته اخلاق غیرقانونی است 
url:  https://www.farsnews.ir/news/14001123000947/عربشاهی-تصمیم-هیات-رئیسه-برای-برکناری-عزیزی-خادم-جهادی-بود-فعالیت-
------------
6690 title:  امیدواری ملی پوش سابق فوتبال ساحلی بابت تغییرات در کادرفنی تیم ملی 
url:  https://www.farsnews.ir/news/14000927000275/امیدوار